In [39]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score

In [16]:
data = pd.read_csv("most_updated_data.csv")
data

,name,element,opponent_team,season,kickoff_time,position,assists,club,clean_sheets,goals_scored,ict_index,minutes,red_cards,saves,selected,result,transfers_balance,was_home,opponent_team_name,total_points
0,Aaron Cresswell,454,4,2016/2017,2016-08-15 19:00:00+00:00,Defender,0,West Ham United,0,0,0.0,0,0,0,14023,L,0,False,Chelsea,0
1,Aaron Lennon,142,17,2016/2017,2016-08-13 14:00:00+00:00,Midfielder,0,Everton,0,0,0.9,15,0,0,13918,D,0,True,Tottenham Hotspur,1
2,Abdoulaye Doucouré,482,13,2016/2017,2016-08-13 14:00:00+00:00,Midfielder,0,Watford,0,0,0.0,0,0,0,1051,D,0,False,Southampton,0
3,Adam Forshaw,286,14,2016/2017,2016-08-13 14:00:00+00:00,Midfielder,0,Middlesbrough,0,0,0.3,69,0,0,2723,D,0,True,Stoke City,1
4,Adam Lallana,205,1,2016/2017,2016-08-14 15:00:00+00:00,Midfielder,1,Liverpool,0,1,14.2,75,0,0,155525,W,0,False,Arsenal,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108678,Matt Doherty,598,19,2023/2024,2023-12-17 14:00:00+00:00,Defender,0,Wolverhampton Wanderers,0,0,0.3,10,0,0,18766,L,-336,False,West Ham United,1
108679,Santiago Bueno,697,19,2023/2024,2023-12-17 14:00:00+00:00,Defender,0,Wolverhampton Wanderers,0,0,0.0,0,0,0,3062,L,-49,False,West Ham United,0
108680,Enso González,698,19,2023/2024,2023-12-17 14:00:00+00:00,Forward,0,Wolverhampton Wanderers,0,0,0.0,0,0,0,1360,L,-50,False,West Ham United,0
108681,Nathan Fraser,704,19,2023/2024,2023-12-17 14:00:00+00:00,Forward,0,Wolverhampton Wanderers,0,0,0.0,0,0,0,7098,L,161,False,West Ham United,0


In [17]:
club_encoder = LabelEncoder()
opp_encoder = LabelEncoder()
pos_encoder = LabelEncoder()
data["club_encoded"] = club_encoder.fit_transform(data["club"])
data["opponent_encoded"] = opp_encoder.fit_transform(data["opponent_team_name"])
data["position_encoded"] = pos_encoder.fit_transform(data["position"])
result_mapping = {'W': 2, 'D': 1, 'L': 0}
data['result_encoded'] = data['result'].map(result_mapping)
data

,name,element,opponent_team,season,kickoff_time,position,assists,club,clean_sheets,goals_scored,...,selected,result,transfers_balance,was_home,opponent_team_name,total_points,club_encoded,opponent_encoded,position_encoded,result_encoded
0,Aaron Cresswell,454,4,2016/2017,2016-08-15 19:00:00+00:00,Defender,0,West Ham United,0,0,...,14023,L,0,False,Chelsea,0,30,6,0,0
1,Aaron Lennon,142,17,2016/2017,2016-08-13 14:00:00+00:00,Midfielder,0,Everton,0,0,...,13918,D,0,True,Tottenham Hotspur,1,8,27,3,1
2,Abdoulaye Doucouré,482,13,2016/2017,2016-08-13 14:00:00+00:00,Midfielder,0,Watford,0,0,...,1051,D,0,False,Southampton,0,28,23,3,1
3,Adam Forshaw,286,14,2016/2017,2016-08-13 14:00:00+00:00,Midfielder,0,Middlesbrough,0,0,...,2723,D,0,True,Stoke City,1,18,24,3,1
4,Adam Lallana,205,1,2016/2017,2016-08-14 15:00:00+00:00,Midfielder,1,Liverpool,0,1,...,155525,W,0,False,Arsenal,11,14,0,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108678,Matt Doherty,598,19,2023/2024,2023-12-17 14:00:00+00:00,Defender,0,Wolverhampton Wanderers,0,0,...,18766,L,-336,False,West Ham United,1,31,30,0,0
108679,Santiago Bueno,697,19,2023/2024,2023-12-17 14:00:00+00:00,Defender,0,Wolverhampton Wanderers,0,0,...,3062,L,-49,False,West Ham United,0,31,30,0,0
108680,Enso González,698,19,2023/2024,2023-12-17 14:00:00+00:00,Forward,0,Wolverhampton Wanderers,0,0,...,1360,L,-50,False,West Ham United,0,31,30,1,0
108681,Nathan Fraser,704,19,2023/2024,2023-12-17 14:00:00+00:00,Forward,0,Wolverhampton Wanderers,0,0,...,7098,L,161,False,West Ham United,0,31,30,1,0


In [19]:
players_grp = data.groupby(["name"])
haaland = players_grp.get_group("Erling Haaland")
haaland.tail(10)

,name,element,opponent_team,season,kickoff_time,position,assists,club,clean_sheets,goals_scored,...,selected,result,transfers_balance,was_home,opponent_team_name,total_points,club_encoded,opponent_encoded,position_encoded,result_encoded
102764,Erling Haaland,355,5,2023/2024,2023-10-21 14:00:00+00:00,Forward,0,Manchester City,0,1,...,8880074,W,-195755,True,Brighton & Hove Albion,8,16,4,1,2
102765,Erling Haaland,355,14,2023/2024,2023-10-29 15:30:00+00:00,Forward,1,Manchester City,1,2,...,8540943,W,-351614,False,Manchester United,16,16,17,1,2
102766,Erling Haaland,355,3,2023/2024,2023-11-04 15:00:00+00:00,Forward,0,Manchester City,0,0,...,8693052,W,86091,True,Bournemouth,1,16,2,1,2
102767,Erling Haaland,355,7,2023/2024,2023-11-12 16:30:00+00:00,Forward,1,Manchester City,0,2,...,8445429,D,-239590,False,Chelsea,16,16,6,1,1
102768,Erling Haaland,355,11,2023/2024,2023-11-25 12:30:00+00:00,Forward,0,Manchester City,0,1,...,8444738,D,-32210,True,Liverpool,7,16,14,1,1
102769,Erling Haaland,355,18,2023/2024,2023-12-03 16:30:00+00:00,Forward,2,Manchester City,0,0,...,8677883,D,178999,True,Tottenham Hotspur,7,16,27,1,1
102770,Erling Haaland,355,2,2023/2024,2023-12-06 20:15:00+00:00,Forward,0,Manchester City,0,0,...,8714407,L,30222,False,Aston Villa,2,16,1,1,0
102771,Erling Haaland,355,12,2023/2024,2023-12-10 14:00:00+00:00,Forward,0,Manchester City,0,0,...,8812520,W,73122,False,Luton Town,0,16,15,1,2
107571,Erling Haaland,355,8,2023/2024,2023-12-16 15:00:00+00:00,Forward,0,Manchester City,0,0,...,7231415,D,-1477506,True,Crystal Palace,0,16,7,1,1
108368,Erling Haaland,355,8,2023/2024,2023-12-16 15:00:00+00:00,Forward,0,Manchester City,0,0,...,7231415,D,-1477506,True,Crystal Palace,0,16,7,1,1


In [20]:
def rolling_averages(group, cols, new_cols):
    # sort data by date, ascending
    group = group.sort_values(["kickoff_time"])
    # compute rolling average - note that `closed = left` means that future data isn't used
    rolling_stats = group[cols].rolling(window = 3, closed = "left").mean()
    # assign the rolling statistics back to the original dataframe
    group[new_cols] = rolling_stats
    # drop the observations containing missing values to prevent propagation of NaN values
    group = group.dropna(subset = new_cols)
    return group

In [21]:
# the cols we want to compute rolling averages for 
cols = ["assists", "clean_sheets", "goals_scored", "ict_index", "minutes", "red_cards", "saves", "selected", "transfers_balance", "result_encoded"]
# append "_rolling" onto each column name
new_cols = [f"{col}_rolling" for col in cols]

In [22]:
rolling_averages(haaland, cols, new_cols).head(20)

,name,element,opponent_team,season,kickoff_time,position,assists,club,clean_sheets,goals_scored,...,assists_rolling,clean_sheets_rolling,goals_scored_rolling,ict_index_rolling,minutes_rolling,red_cards_rolling,saves_rolling,selected_rolling,transfers_balance_rolling,result_encoded_rolling
70085,Erling Haaland,318,7,2022/2023,2022-08-27 14:00:00+00:00,Forward,0,Manchester City,0,3,...,0.333333,0.666667,1.000000,10.666667,80.000000,0.0,0.0,4.767158e+06,447084.666667,1.666667
70672,Erling Haaland,318,16,2022/2023,2022-08-31 18:30:00+00:00,Forward,0,Manchester City,1,3,...,0.333333,0.333333,1.333333,12.466667,82.000000,0.0,0.0,5.623560e+06,480293.000000,1.666667
71264,Erling Haaland,318,2,2022/2023,2022-09-03 16:30:00+00:00,Forward,0,Manchester City,0,1,...,0.000000,0.333333,2.333333,16.933333,80.333333,0.0,0.0,6.146089e+06,317286.000000,1.666667
71846,Erling Haaland,318,20,2022/2023,2022-09-17 11:30:00+00:00,Forward,0,Manchester City,1,1,...,0.000000,0.333333,2.333333,17.266667,80.333333,0.0,0.0,6.847716e+06,502273.333333,1.666667
72303,Erling Haaland,318,14,2022/2023,2022-10-02 13:00:00+00:00,Forward,2,Manchester City,0,3,...,0.000000,0.666667,1.666667,12.900000,82.666667,0.0,0.0,7.660872e+06,514079.000000,1.666667
72925,Erling Haaland,318,17,2022/2023,2022-10-08 14:00:00+00:00,Forward,0,Manchester City,1,1,...,0.666667,0.333333,1.666667,16.766667,90.000000,0.0,0.0,8.245741e+06,337170.333333,1.666667
73549,Erling Haaland,318,12,2022/2023,2022-10-16 15:30:00+00:00,Forward,0,Manchester City,0,0,...,0.666667,0.666667,1.666667,16.533333,90.000000,0.0,0.0,8.620385e+06,167291.666667,2.000000
74745,Erling Haaland,318,5,2022/2023,2022-10-22 14:00:00+00:00,Forward,0,Manchester City,0,2,...,0.666667,0.333333,1.333333,19.566667,90.000000,0.0,0.0,8.814516e+06,135821.666667,1.333333
75377,Erling Haaland,318,10,2022/2023,2022-10-29 11:30:00+00:00,Forward,0,Manchester City,0,0,...,0.000000,0.333333,1.000000,13.933333,86.333333,0.0,0.0,8.959631e+06,218493.000000,1.333333
76017,Erling Haaland,318,9,2022/2023,2022-11-05 15:00:00+00:00,Forward,0,Manchester City,0,1,...,0.000000,0.000000,0.666667,9.666667,56.333333,0.0,0.0,8.907564e+06,44174.333333,1.333333


In [23]:
player_rolling = data.groupby("name").apply(lambda x: rolling_averages(x, cols, new_cols))

In [24]:
player_rolling

name  element  opponent_team     season  \
name                                                                      
Aaron Connolly 21090  Aaron Connolly       78              7  2020/2021   
               21641  Aaron Connolly       78              6  2020/2021   
               22217  Aaron Connolly       78             18  2020/2021   
               22795  Aaron Connolly       78             17  2020/2021   
               23378  Aaron Connolly       78              4  2020/2021   
...                              ...      ...            ...        ...   
Ørjan Nyland   40661    Ørjan Nyland       35             13  2020/2021   
               40662    Ørjan Nyland       35              7  2020/2021   
               41532    Ørjan Nyland       35              6  2020/2021   
               42126    Ørjan Nyland       35             17  2020/2021   
               42811    Ørjan Nyland       35              5  2020/2021   

                                   kickoff_time    position  assists  \
name                                                                   
Aaron Connolly 21090  2020-10-03 14:00:00+00:00     Forward        0   
               21641  2020-10-18 13:00:00+00:00     Forward        1   
               22217  2020-10-26 17:30:00+00:00     Forward        0   
               22795  2020-11-01 19:15:00+00:00     Forward        0   
               23378  2020-11-06 17:30:00+00:00     Forward        0   
...                                         ...         ...      ...   
Ørjan Nyland   40661  2021-05-09 13:05:00+00:00  Goalkeeper        0   
               40662  2021-05-13 17:00:00+00:00  Goalkeeper        0   
               41532  2021-05-16 11:00:00+00:00  Goalkeeper        0   
               42126  2021-05-19 17:00:00+00:00  Goalkeeper        0   
               42811  2021-05-23 15:00:00+00:00  Goalkeeper        0   

                                        club  clean_sheets  goals_scored  ...  \
name                                                                      ...   
Aaron Connolly 21090  Brighton & Hove Albion             0             0  ...   
               21641  Brighton & Hove Albion             0             0  ...   
               22217  Brighton & Hove Albion             0             0  ...   
               22795  Brighton & Hove Albion             0             0  ...   
               23378  Brighton & Hove Albion             0             0  ...   
...                                      ...           ...           ...  ...   
Ørjan Nyland   40661             Aston Villa             0             0  ...   
               40662             Aston Villa             0             0  ...   
               41532             Aston Villa             0             0  ...   
               42126             Aston Villa             0             0  ...   
               42811             Aston Villa             0             0  ...   

                      assists_rolling  clean_sheets_rolling  \
name                                                          
Aaron Connolly 21090         0.000000              0.333333   
               21641         0.000000              0.333333   
               22217         0.333333              0.000000   
               22795         0.333333              0.000000   
               23378         0.333333              0.000000   
...                               ...                   ...   
Ørjan Nyland   40661         0.000000              0.000000   
               40662         0.000000              0.000000   
               41532         0.000000              0.000000   
               42126         0.000000              0.000000   
               42811         0.000000              0.000000   

                      goals_scored_rolling  ict_index_rolling  \
name                                                            
Aaron Connolly 21090              0.333333           4.066667   
               21641              0.333333        

In [11]:
player_rolling = player_rolling.droplevel("name")
player_rolling

,name,element,opponent_team,season,kickoff_time,position,assists,club,clean_sheets,goals_scored,...,assists_rolling,clean_sheets_rolling,goals_scored_rolling,ict_index_rolling,minutes_rolling,red_cards_rolling,saves_rolling,selected_rolling,transfers_balance_rolling,result_encoded_rolling
21090,Aaron Connolly,78,7,2020/2021,2020-10-03 14:00:00+00:00,Forward,0,Brighton & Hove Albion,0,0,...,0.000000,0.333333,0.333333,4.066667,69.000000,0.0,0.0,40863.666667,4121.666667,0.666667
21641,Aaron Connolly,78,6,2020/2021,2020-10-18 13:00:00+00:00,Forward,1,Brighton & Hove Albion,0,0,...,0.000000,0.333333,0.333333,3.166667,75.666667,0.0,0.0,48503.666667,3684.666667,0.666667
22217,Aaron Connolly,78,18,2020/2021,2020-10-26 17:30:00+00:00,Forward,0,Brighton & Hove Albion,0,0,...,0.333333,0.000000,0.000000,1.866667,50.000000,0.0,0.0,52418.333333,1074.333333,0.333333
22795,Aaron Connolly,78,17,2020/2021,2020-11-01 19:15:00+00:00,Forward,0,Brighton & Hove Albion,0,0,...,0.333333,0.000000,0.000000,1.233333,25.666667,0.0,0.0,50905.666667,-2750.000000,0.666667
23378,Aaron Connolly,78,4,2020/2021,2020-11-06 17:30:00+00:00,Forward,0,Brighton & Hove Albion,0,0,...,0.333333,0.000000,0.000000,1.000000,4.000000,0.0,0.0,47046.333333,-4661.333333,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40661,Ørjan Nyland,35,13,2020/2021,2021-05-09 13:05:00+00:00,Goalkeeper,0,Aston Villa,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,281394.333333,-2122.333333,1.000000
40662,Ørjan Nyland,35,7,2020/2021,2021-05-13 17:00:00+00:00,Goalkeeper,0,Aston Villa,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,278999.000000,-2346.666667,1.000000
41532,Ørjan Nyland,35,6,2020/2021,2021-05-16 11:00:00+00:00,Goalkeeper,0,Aston Villa,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,277527.000000,-2471.000000,1.000000
42126,Ørjan Nyland,35,17,2020/2021,2021-05-19 17:00:00+00:00,Goalkeeper,0,Aston Villa,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,276053.000000,-2426.000000,0.333333


In [25]:
player_rolling.index = range(player_rolling.shape[0])
player_rolling

,name,element,opponent_team,season,kickoff_time,position,assists,club,clean_sheets,goals_scored,...,assists_rolling,clean_sheets_rolling,goals_scored_rolling,ict_index_rolling,minutes_rolling,red_cards_rolling,saves_rolling,selected_rolling,transfers_balance_rolling,result_encoded_rolling
0,Aaron Connolly,78,7,2020/2021,2020-10-03 14:00:00+00:00,Forward,0,Brighton & Hove Albion,0,0,...,0.000000,0.333333,0.333333,4.066667,69.000000,0.0,0.0,40863.666667,4121.666667,0.666667
1,Aaron Connolly,78,6,2020/2021,2020-10-18 13:00:00+00:00,Forward,1,Brighton & Hove Albion,0,0,...,0.000000,0.333333,0.333333,3.166667,75.666667,0.0,0.0,48503.666667,3684.666667,0.666667
2,Aaron Connolly,78,18,2020/2021,2020-10-26 17:30:00+00:00,Forward,0,Brighton & Hove Albion,0,0,...,0.333333,0.000000,0.000000,1.866667,50.000000,0.0,0.0,52418.333333,1074.333333,0.333333
3,Aaron Connolly,78,17,2020/2021,2020-11-01 19:15:00+00:00,Forward,0,Brighton & Hove Albion,0,0,...,0.333333,0.000000,0.000000,1.233333,25.666667,0.0,0.0,50905.666667,-2750.000000,0.666667
4,Aaron Connolly,78,4,2020/2021,2020-11-06 17:30:00+00:00,Forward,0,Brighton & Hove Albion,0,0,...,0.333333,0.000000,0.000000,1.000000,4.000000,0.0,0.0,47046.333333,-4661.333333,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104234,Ørjan Nyland,35,13,2020/2021,2021-05-09 13:05:00+00:00,Goalkeeper,0,Aston Villa,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,281394.333333,-2122.333333,1.000000
104235,Ørjan Nyland,35,7,2020/2021,2021-05-13 17:00:00+00:00,Goalkeeper,0,Aston Villa,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,278999.000000,-2346.666667,1.000000
104236,Ørjan Nyland,35,6,2020/2021,2021-05-16 11:00:00+00:00,Goalkeeper,0,Aston Villa,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,277527.000000,-2471.000000,1.000000
104237,Ørjan Nyland,35,17,2020/2021,2021-05-19 17:00:00+00:00,Goalkeeper,0,Aston Villa,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,276053.000000,-2426.000000,0.333333


In [26]:
player_rolling = player_rolling.drop_duplicates()
player_rolling

,name,element,opponent_team,season,kickoff_time,position,assists,club,clean_sheets,goals_scored,...,assists_rolling,clean_sheets_rolling,goals_scored_rolling,ict_index_rolling,minutes_rolling,red_cards_rolling,saves_rolling,selected_rolling,transfers_balance_rolling,result_encoded_rolling
0,Aaron Connolly,78,7,2020/2021,2020-10-03 14:00:00+00:00,Forward,0,Brighton & Hove Albion,0,0,...,0.000000,0.333333,0.333333,4.066667,69.000000,0.0,0.0,40863.666667,4121.666667,0.666667
1,Aaron Connolly,78,6,2020/2021,2020-10-18 13:00:00+00:00,Forward,1,Brighton & Hove Albion,0,0,...,0.000000,0.333333,0.333333,3.166667,75.666667,0.0,0.0,48503.666667,3684.666667,0.666667
2,Aaron Connolly,78,18,2020/2021,2020-10-26 17:30:00+00:00,Forward,0,Brighton & Hove Albion,0,0,...,0.333333,0.000000,0.000000,1.866667,50.000000,0.0,0.0,52418.333333,1074.333333,0.333333
3,Aaron Connolly,78,17,2020/2021,2020-11-01 19:15:00+00:00,Forward,0,Brighton & Hove Albion,0,0,...,0.333333,0.000000,0.000000,1.233333,25.666667,0.0,0.0,50905.666667,-2750.000000,0.666667
4,Aaron Connolly,78,4,2020/2021,2020-11-06 17:30:00+00:00,Forward,0,Brighton & Hove Albion,0,0,...,0.333333,0.000000,0.000000,1.000000,4.000000,0.0,0.0,47046.333333,-4661.333333,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104234,Ørjan Nyland,35,13,2020/2021,2021-05-09 13:05:00+00:00,Goalkeeper,0,Aston Villa,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,281394.333333,-2122.333333,1.000000
104235,Ørjan Nyland,35,7,2020/2021,2021-05-13 17:00:00+00:00,Goalkeeper,0,Aston Villa,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,278999.000000,-2346.666667,1.000000
104236,Ørjan Nyland,35,6,2020/2021,2021-05-16 11:00:00+00:00,Goalkeeper,0,Aston Villa,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,277527.000000,-2471.000000,1.000000
104237,Ørjan Nyland,35,17,2020/2021,2021-05-19 17:00:00+00:00,Goalkeeper,0,Aston Villa,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,276053.000000,-2426.000000,0.333333


In [27]:
def make_predictions(data, predictors): 
    # split data
    train = data[data["date"] < "2023-01-01"]
    test = train = data[data["date"] > "2023-01-01"]
    # fit random forest model
    rf.fit(train[predictors], train["targets"])
    # create predictions
    preds = rf.predict(test[predictors])
    # combine predictions with actual results
    combined = pd.DataFrame(dict(actual = test["targets"], prediction = preds), index = test.index)
    # calculate precision
    precision = precision_score(test["targets"], preds)
    return combined, precision

In [28]:
features = ["position_encoded", "club_encoded", "was_home", "club_encoded", "opponent_encoded", "position_encoded", 
            "assists_rolling", "clean_sheets_rolling", "goals_scored_rolling", "ict_index_rolling", "minutes_rolling", 
            "red_cards_rolling", "saves_rolling", "selected_rolling", "transfers_balance_rolling", "result_encoded_rolling"]
target = ["total_points"]

In [30]:
seasons = np.unique(player_rolling["season"])
train_seasons = seasons[:4]
val_season = seasons[4]
test_seasons = seasons[5:]
train = player_rolling[player_rolling["season"].isin(train_seasons)]
val = player_rolling[player_rolling["season"] == val_season]
test = player_rolling[player_rolling["season"].isin(test_seasons)]
train_seasons, val_season, test_seasons, train.shape, val.shape, test.shape

(array(['2016/2017', '2017/2018', '2020/2021', '2021/2022'], dtype=object),
 '2022/2023',
 array(['2023/2024'], dtype=object),
 (65390, 34),
 (24827, 34),
 (14019, 34))

In [35]:
# hyperparameter tuning: grid search
param_dist = {
    "n_estimators": [100, 200, 300, 400],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10, 15],
    "min_samples_leaf": [1, 2, 4, 8]
}

param_dist2 = {
    "n_estimators": [300, 400, 500, 600],
    "max_depth": [10, 20],
    "min_samples_split": [10, 15, 20, 25, 30],
    "min_samples_leaf": [4, 8, 10, 12]
}

rf_tuning = RandomForestRegressor()

grid_search = GridSearchCV(estimator=rf_tuning, 
                           param_grid=param_dist2, 
                           cv=5, 
                           scoring='neg_mean_squared_error', 
                           n_jobs=-1)
grid_search.fit(val[features], np.ravel(val[target]))

best_params = grid_search.best_params_
best_params

{'max_depth': 10,
 'min_samples_leaf': 12,
 'min_samples_split': 30,
 'n_estimators': 600}

In [70]:
rf_reg = RandomForestRegressor(max_depth=10, min_samples_leaf=200, min_samples_split=200, n_estimators=1200, n_jobs=-1, random_state=21)
rf_reg.fit(train[features], np.ravel(train[target]))

RandomForestRegressor(max_depth=10, min_samples_leaf=200, min_samples_split=200,
                      n_estimators=1200, n_jobs=-1, random_state=21)

In [71]:
predictions = rf_reg.predict(test[features])

In [72]:
mae = mean_absolute_error(test[target], predictions)
mse = mean_squared_error(test[target], predictions)
rmse = mean_squared_error(test[target], predictions, squared=False)  # Pass squared=False to get RMSE
r2 = r2_score(test[target], predictions)
explained_variance = explained_variance_score(test[target], predictions)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2 Score: {r2}')
print(f'Explained Variance: {explained_variance}')

MAE: 1.0742549637706398
MSE: 3.963816862977304
RMSE: 1.9909336661419195
R2 Score: 0.2944002711267686
Explained Variance: 0.2947969101752913


In [41]:
predictions

array([0.07686528, 0.10554324, 0.14025626, ..., 0.02037136, 0.0207938 ,
       0.02069784])

In [46]:
test_copy = test.copy()
test_copy["predictions"] = predictions
first_test_df = test_copy[["name", "club", "opponent_team", "kickoff_time", "result", "total_points", "predictions"]]
first_test_df

,name,club,opponent_team,kickoff_time,result,total_points,predictions
73,Aaron Connolly,Brighton & Hove Albion,12,2023-08-12 14:00:00+00:00,W,0,0.076865
74,Aaron Connolly,Brighton & Hove Albion,20,2023-08-19 14:00:00+00:00,W,0,0.105543
75,Aaron Connolly,Brighton & Hove Albion,19,2023-08-26 16:30:00+00:00,L,0,0.140256
76,Aaron Connolly,Brighton & Hove Albion,15,2023-09-02 16:30:00+00:00,W,0,0.192282
77,Aaron Connolly,Brighton & Hove Albion,14,2023-09-16 14:00:00+00:00,W,0,0.219731
...,...,...,...,...,...,...,...
104088,Ângelo Gabriel Borges Damaceno,Chelsea,5,2023-12-03 14:00:00+00:00,W,0,0.023506
104089,Ângelo Gabriel Borges Damaceno,Chelsea,14,2023-12-06 20:15:00+00:00,L,0,0.024140
104090,Ângelo Gabriel Borges Damaceno,Chelsea,9,2023-12-10 14:00:00+00:00,L,0,0.020371
104091,Ângelo Gabriel Borges Damaceno,Chelsea,17,2023-12-16 15:00:00+00:00,W,0,0.020794


In [48]:
first_test_df.to_csv("first_test_df.csv", index=False)

In [54]:
dict(zip(features, rf_reg.feature_importances_))

{'position_encoded': 0.0035102256222629734,
 'club_encoded': 0.01318611256246012,
 'was_home': 0.007967790092117464,
 'opponent_encoded': 0.04006843701989891,
 'assists_rolling': 0.004082005551625065,
 'clean_sheets_rolling': 0.007038568373727562,
 'goals_scored_rolling': 0.003586191676213931,
 'ict_index_rolling': 0.2479858928138833,
 'minutes_rolling': 0.49313874575030636,
 'red_cards_rolling': 0.0001919070894073952,
 'saves_rolling': 0.005157550110070596,
 'selected_rolling': 0.0912541756546442,
 'transfers_balance_rolling': 0.051541753314292456,
 'result_encoded_rolling': 0.014610067881761234}